In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime 

In [2]:
def proper_index(df):
    df['datetime'] = pd.to_datetime(df['datetime_beginning_ept'])
    df['datetime'] = df.datetime.dt.strftime('%m/%d/%y %H:%M')
    df = df.set_index('datetime')
    df = df.drop(['datetime_beginning_utc', 'datetime_beginning_ept'], axis=1)
    return df

def missing_values(df):
    missing = pd.concat([df.isnull().sum(), 100 * df.isnull().mean()], axis=1)
    missing.columns=(['count', '%'])
    missing.sort_values(by='count', ascending=False)
    return missing 

In [3]:
pepco_july = pd.read_csv('../Pepco/rt_hrl_lmps_jul_pepco.csv')
pepco_aug = pd.read_csv('../Pepco/rt_hrl_lmps_aug_pepco.csv')
pepco_sep =pd.read_csv('../Pepco/rt_hrl_lmps_sep_pepco.csv')

pnode_state = pd.read_excel('../pnode-by-state.xls', header=6)

solar = pd.read_csv('../solar_gen.csv')
wind = pd.read_csv('../wind_gen.csv')

fuel_aug=pd.read_csv('../Fuel Type/gen_by_fuel_aug.csv')
fuel_july=pd.read_csv('../Fuel Type/gen_by_fuel_jul.csv')
fuel_sep=pd.read_csv('../Fuel Type/gen_by_fuel_sep.csv')

In [4]:
pepco_df = pd.concat([pepco_july, pepco_aug, pepco_sep])

fuel = pd.concat([fuel_july, fuel_aug, fuel_sep])

In [5]:
#the only areas that are untilities operate in are MIDATL & RTO
area = ['MIDATL', 'RTO']

wind['area'].unique()
wind = wind[wind.area.isin(area)]
wind.head()


solar['area'].unique()
solar = solar[solar.area.isin(area)]
solar.head()

print(list(wind.area.unique()))
print(list(solar.area.unique()))

['MIDATL', 'RTO']
['MIDATL', 'RTO']


In [6]:
pepco = proper_index(pepco_df)
pepco = pepco.sort_index()



solar = proper_index(solar)
solar = solar.sort_index()
wind = proper_index(wind)
wind = wind.sort_index()

fuel = proper_index(fuel)
fuel = fuel.sort_index()

In [7]:
pepco = pd.merge(pepco, wind, how='left', left_index=True, right_index=True)
pepco = pd.merge(pepco, solar, how='left', left_index=True, right_index=True)



In [8]:
pepco.dtypes

pnode_id                    int64
pnode_name                 object
voltage                    object
equipment                  object
type                       object
zone                       object
system_energy_price_rt    float64
total_lmp_rt              float64
congestion_price_rt       float64
marginal_loss_price_rt    float64
row_is_current               bool
version_nbr                 int64
area_x                     object
wind_generation_mw        float64
area_y                     object
solar_generation_mw       float64
dtype: object

In [9]:
pepco =  pd.merge(pepco, fuel, how='left', left_index=True, right_index=True)
pepco.head()

,pnode_id,pnode_name,voltage,equipment,type,zone,system_energy_price_rt,total_lmp_rt,congestion_price_rt,marginal_loss_price_rt,row_is_current,version_nbr,area_x,wind_generation_mw,area_y,solar_generation_mw,fuel_type,mw,fuel_percentage_of_total,is_renewable
datetime,,,,,,,,,,,,,,,,,,,,
07/01/20 00:00,49701,BELLSMIL,34 KV,LD1B,LOAD,PEPCO,15.65,15.78,0.0,0.12,True,1,RTO,549.993,RTO,-2.46,Wind,536.1,0.01,True
07/01/20 00:00,49701,BELLSMIL,34 KV,LD1B,LOAD,PEPCO,15.65,15.78,0.0,0.12,True,1,RTO,549.993,RTO,-2.46,Storage,0.0,0.00,False
07/01/20 00:00,49701,BELLSMIL,34 KV,LD1B,LOAD,PEPCO,15.65,15.78,0.0,0.12,True,1,RTO,549.993,RTO,-2.46,Solar,21.9,0.00,True
07/01/20 00:00,49701,BELLSMIL,34 KV,LD1B,LOAD,PEPCO,15.65,15.78,0.0,0.12,True,1,RTO,549.993,RTO,-2.46,Other Renewables,629.3,0.01,True
07/01/20 00:00,49701,BELLSMIL,34 KV,LD1B,LOAD,PEPCO,15.65,15.78,0.0,0.12,True,1,RTO,549.993,RTO,-2.46,Other,37.4,0.00,False


In [10]:
#pepco2 = pd.merge(pepco, pnode_state, how='inner', on='pnode_id')
pnode_state = pnode_state.rename({'PNODEID':'pnode_id'}, axis=1)

In [11]:
pepco['Date-Time'] = pepco.index
pepco3 = pd.merge(pepco, pnode_state, how='inner', on='pnode_id')


In [12]:
#pepco3['datetime'] = pd.to_datetime(pepco3['Date-Time'])
#pepco3['datetime'] = pepco3.datetime.dt.strftime('%m/%d/%y %H:%M')
#pepco3 = pepco3.set_index('datetime')


In [14]:
pepco3 = pepco3.drop(['pnode_name', 'type','Date-Time'], axis=1)

MemoryError: Unable to allocate 1.20 GiB for an array with shape (12, 13419252) and data type object